In [33]:
from pymongo import MongoClient
import requests
from IPython.display import display, HTML, JSON
import math
import re
import pandas as pd

In [2]:
def parse_doi(doi):
    return re.sub(r'https*\:\/\/[\w\.]+\/','',doi.lower())

In [3]:
sample_db = MongoClient()["kahi_sample_works"]

In [4]:
url="https://inspirehep.net/api/literature?q=a+restrepo,+d&earliest_date=2017--2019&doc_type=published"


In [5]:
req=requests.get(url)

In [6]:
dois_data = req.json()

In [7]:
JSON(dois_data["hits"]["hits"][0]["metadata"]["dois"])

<IPython.core.display.JSON object>

In [8]:
dois=[]
for i in dois_data["hits"]["hits"]:
    doi=i["metadata"]["dois"][0]["value"]
    dois.append(doi)

In [9]:
dois

['10.1103/PhysRevD.98.055016',
 '10.1103/PhysRevD.100.035029',
 '10.1016/j.physletb.2018.01.009',
 '10.1103/PhysRevD.99.015038',
 '10.1016/j.physletb.2019.01.023',
 '10.1103/PhysRevD.97.055045',
 '10.1103/PhysRevD.97.115032',
 '10.1103/PhysRevD.101.035004',
 '10.1007/JHEP04(2020)134',
 '10.1142/S0217732318500244']

# openalex

In [10]:
oa_db = MongoClient()["openalexco"]
oa_works = MongoClient()["openalexco"]["works"]

In [11]:
sample=[]
for i in dois:
    doi=i.lower() #openalex requires lower case
    prefix="https://doi.org/"
    found = oa_works.find_one({"doi":prefix+doi})
    if found is not None:
        print(prefix+doi)
        sample.append(found)
    else:
        pass
        #print(prefix+doi)

https://doi.org/10.1103/physrevd.98.055016
https://doi.org/10.1103/physrevd.100.035029
https://doi.org/10.1016/j.physletb.2018.01.009
https://doi.org/10.1103/physrevd.99.015038
https://doi.org/10.1016/j.physletb.2019.01.023
https://doi.org/10.1103/physrevd.97.055045
https://doi.org/10.1103/physrevd.97.115032
https://doi.org/10.1103/physrevd.101.035004
https://doi.org/10.1007/jhep04(2020)134
https://doi.org/10.1142/s0217732318500244


In [12]:
len(sample)

10

In [13]:

sample_db["oa_works"].drop()
sample_db["oa_works"].insert_many(sample)

InsertManyResult([ObjectId('637cfba0244b9f9d0fe13cd6'), ObjectId('637cfb9d244b9f9d0fe0e8c2'), ObjectId('637cfba2244b9f9d0fe1641d'), ObjectId('637cfb9f244b9f9d0fe113f2'), ObjectId('637cfb9f244b9f9d0fe11528'), ObjectId('637cfba2244b9f9d0fe161bb'), ObjectId('637cfba1244b9f9d0fe151d2'), ObjectId('637cfb9b244b9f9d0fe0ba80'), ObjectId('637cfb9a244b9f9d0fe0b0c4'), ObjectId('637cfba3244b9f9d0fe16be0')], acknowledged=True)

# Scopus

In [14]:
sc_db = MongoClient()["scopus_colombia"]
sc_stage = sc_db["stage"]

In [15]:
sample=[]
for doi in dois:
    found  = sc_stage.find_one({"DOI":doi.upper()})
    if found is not None:
        sample.append(found)
    else:
        print(doi)

10.1103/PhysRevD.98.055016
10.1103/PhysRevD.100.035029
10.1016/j.physletb.2018.01.009
10.1103/PhysRevD.99.015038
10.1016/j.physletb.2019.01.023
10.1103/PhysRevD.97.055045
10.1103/PhysRevD.97.115032
10.1103/PhysRevD.101.035004


In [16]:
len(sample)

2

In [17]:
sample_db["scopus_works"].drop()
sample_db["scopus_works"].insert_many(sample)

InsertManyResult([ObjectId('5f86370592dec2f3a4e2af09'), ObjectId('5f86377b92dec2f3a4e3358d')], acknowledged=True)

In [18]:
#double checking if only to dois found
data=list(sc_stage.find({"DOI":{"$ne":math.nan}},{"DOI":1}))
for doi in dois:
    for rec in data:
        sdoi=rec["DOI"].lower().strip()
        if doi == sdoi:
            print("found"+doi)
            break


found10.1016/j.physletb.2018.01.009
found10.1016/j.physletb.2019.01.023


# WOS

In [19]:
wos_db = MongoClient()["wos_colombia"]
wos_stage = wos_db["stage"]

In [20]:
sample=[]
for doi in dois:
    found  = wos_stage.find_one({"doi_idx":doi.lower()})
    if found is not None:
        sample.append(found)
    else:
        print(doi)

10.1007/JHEP04(2020)134


In [21]:
len(sample)

9

In [22]:
sample_db["wos_works"].drop()
sample_db["wos_works"].insert_many(sample)

InsertManyResult([ObjectId('5f86270892dec2f3a4dfa333'), ObjectId('5f8625b392dec2f3a4df7596'), ObjectId('5f86279292dec2f3a4dfb4ec'), ObjectId('5f86265f92dec2f3a4df8be8'), ObjectId('5f86264192dec2f3a4df878c'), ObjectId('5f86279292dec2f3a4dfb499'), ObjectId('5f86275492dec2f3a4dfacfd'), ObjectId('5f86251f92dec2f3a4df6218'), ObjectId('5f8627af92dec2f3a4dfb912')], acknowledged=True)

# Scholar

In [23]:
gs_db = MongoClient()["scholar_colombia"]
gs_stage = gs_db["stage"]

In [24]:
sample=[]
for doi in dois:
    found  = gs_stage.find_one({"doi":doi.lower()})
    if found is not None:
        sample.append(found)
    else:
        print(doi)

In [25]:
len(sample)

10

In [26]:
sample_db["scholar_works"].drop()
sample_db["scholar_works"].insert_many(sample)

InsertManyResult([ObjectId('629115516f6a44dc7d6b3e39'), ObjectId('629115506f6a44dc7d69e8c8'), ObjectId('629115506f6a44dc7d69eb12'), ObjectId('629115516f6a44dc7d6b34df'), ObjectId('629115506f6a44dc7d69d281'), ObjectId('629115516f6a44dc7d6b458d'), ObjectId('629115516f6a44dc7d6b428a'), ObjectId('629115516f6a44dc7d6b2393'), ObjectId('629115516f6a44dc7d6bca95'), ObjectId('629115516f6a44dc7d6b4719')], acknowledged=True)

# Scienti

In [27]:
scti_db = MongoClient()["scienti_111"]
scti_stage = scti_db["products"]

In [28]:
scti_dois = list(scti_stage.find({"TXT_DOI":{"$exists":True,"$ne":None}},{"TXT_DOI":1}))

In [29]:
sample_ids=[]
for doi in dois:
    for scdoi in scti_dois:
        sdoi = parse_doi(scdoi["TXT_DOI"])
        if doi.lower() == sdoi:
            sample_ids.append(scdoi["_id"])

In [30]:
len(sample_ids)

8

In [31]:
sample = []
for _id in  sample_ids:
    rec = scti_stage.find_one({"_id":_id})
    sample.append(rec)

In [32]:
sample_db["scienti_works"].drop()
sample_db["scienti_works"].insert_many(sample)

InsertManyResult([ObjectId('637d3a26244b9f9d0fe574ae'), ObjectId('637d3a26244b9f9d0fe574b4'), ObjectId('637d3a26244b9f9d0fe574b2'), ObjectId('637d3a26244b9f9d0fe574b0'), ObjectId('637d3a26244b9f9d0fe574b1'), ObjectId('637d3a28244b9f9d0fe597dc'), ObjectId('637d3a26244b9f9d0fe574b3'), ObjectId('637d3a28244b9f9d0fe597d3')], acknowledged=True)

# TODO Puntaje

In [34]:
path = "~/data/kahi_data/kahi_data/udea/produccion 2018-2022 al 27 oct 2022.xlsx"
df = pd.read_excel(path)

In [35]:
sample=[]
for doi in dois:
    for i, row in df.iterrows():
        if doi == row['DOI']:
            sample.append(row)
len(sample)

8

In [37]:
df_sample = pd.DataFrame(sample)
df_sample.to_excel("puntaje_sample.xlsx", index=False)